In [42]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import pandas as pd
import numpy as np

import inventario_finanzas_int.funciones_auxiliares as fa

pd.set_option('display.max_columns', None)

In [44]:
df_mobiliario = pd.read_csv("../data/processed/df_procesada_mobiliarios.csv")
df_equipos_medicos = pd.read_csv("../data/processed/df_procesada_equipos_medicos.csv")
df_industriales = pd.read_csv("../data/processed/df_procesada_industriales.csv")

In [45]:
union_bienes = pd.concat([df_mobiliario, df_equipos_medicos, df_industriales])

In [46]:
union_bienes["tipo_bien"].value_counts()

tipo_bien
MOBILIARIO           4198
EQUIPO MEDICO         973
EQUIPO INDUSTRIAL     871
Name: count, dtype: int64

In [47]:
union_bienes.query("tipo_bien == 'MOBILIARIO'")["unidadservicio_clinico"].value_counts()

unidadservicio_clinico
HEMODINAMIA                              304
CONSULTORIO EXTERNO                      262
ADMISION                                  74
REHEBILITACION CARDIOPULMONAR             20
CONGENITOS                                19
UPC 5 PISO                                 2
ECOCARDIOGRAFIA                            1
APNEA DEL SUENO                            1
FISIOPATOLOGIA                             1
UMT                                        1
LAVORATORIO                                1
QUIMIOTERAPIA                              1
IMAGENOLOGIA                               1
FARMACIA HOPITALIZADOS                     1
BRONCOSCOPIA                               1
UTIM NORTE                                 1
UCI 5 NORTE                                1
MQ CARDIOVASCULAR                          1
UTI NORTE                                  1
CIRUGIA DE TORAX                           1
UTI SUR                                    1
MQ 3 NORTE                      

In [48]:
union_bienes.query("tipo_bien == 'EQUIPO MEDICO'")["unidadservicio_clinico"].value_counts()

unidadservicio_clinico
UPC 5 NORTE                         160
UTI 4 SUR                           147
MQ 3 SUR                            137
LABORATORIO                         131
MQ 3 NORTE                          121
UTI 3 NORTE                          76
UTI 4 NORTE                          74
CONSULTORIO EXTERNO                  70
FUNCION PULMONAR                     27
UMT                                  20
UNIDAD DE APOYO FARMACIA              7
MEDICINA FISICA Y REHABILITACION      2
CIRUGIA DE TORAX                      1
Name: count, dtype: int64

In [49]:
union_bienes.query("tipo_bien == 'EQUIPO INDUSTRIAL'")["unidadservicio_clinico"].value_counts()

unidadservicio_clinico
MQ 3 SUR                            135
MQ 3 NORTE                          120
UTI 3 NORTE                          80
UPC 5 NORTE                          80
4 MQ CARDIOVASCULAR                  80
UTI 4 NORTE                          72
UTI 4 SUR                            56
LABORATORIO                          50
CONSULTORIO EXTERNO                  39
MEDICINA FISICA Y REHABILITACION     35
ESTERILIZACION                       32
HEMODINAMIA                          30
SALA DE MEDICOS CARDIOCIRUGIA        15
FARMACIA HOSPITALIZADO               12
ANATOMIA PATOLOGICA                  11
ADMISION                             10
CUIDADOS PALIATIVOS                   7
4 NORTE                               6
CONGENITO                             1
Name: count, dtype: int64